## Comorbidity analysis by medications

### ACT ontology mapping
- To calculate the prevalence of various comorbidities in the ASD cohort, we first had to develop a mapping table using the ACT Ontology v2.0.1, which contained ICD-9 and ICD-10 codes.

- 108,024 distinct ICD-9 and ICD-10 codes were mapped to ACT terms aggregated in levels using Microsoft Excel. The LEN function, which is built into Excel, was used to construct the following formula:
    **=LEN(A3)-LEN(SUBSTITUTE(A3,"\",""))**

- When applied to the input table, this formula transformed the ICD-9 and ICD-10 codes into a table organized from Level 0 (the most general category) to Level 9 (the deepest category). ICD-10 codes were organized up to category n, while ICD-9 codes were organized up to category n-1.

- The transformed table was then uploaded into the SQL server and named ACT_ICD10_ICD9_3. 

- To retrieve comorbidities associated with each individual in the ASD cohort and join the ASD cohort with the ACT Ontology mapping table, the ASDMembers, FactICD, and ACT_codes all had to be joined. Since this mapping process was time-intensive, it was performed separately for each level of the mapping table.

### Comorbidity analysis

- In order to estimate the prevalence of comorbidities in the ASD cohort based on the primary medication being taken, the previously mapped ACT table had to also be joined with a table of pharmacy claims.

- First, pharmacy claims for the ASD cohort from 2014-2019 was retrieved.

- The ASD cohort was then divided into subsets based on the medications of interest in this study (e.g., methylphenidate, guanfacine). First, a subset of the cohort was created for individuals taking each drug.


#### Medication Input list
First we read the file that contains all the medications that we are analyzing. 

In [ ]:
library("devtools")
library("SqlServerJtds")
library("SqlTools")
library("FactToCube")
library("ggplot2")
library("plotly")
library("ggalluvial")

In [ ]:
medInputList <- read.delim("./medInputList", header = TRUE, sep = "\t", colClasses = "character")
groups <- as.character( unique( medInputList$Group))

#### Query to identify patients taking only one specific drug

In [ ]:
for( i in 1:length( groups )){
  print(i)
  meds <- paste( tolower(medInputList[ medInputList$Group == groups[i], "medicationName"]), collapse="%' OR LOWER(NdcDescription)  like '%")
  queryStart <- paste0( "SELECT DISTINCT MemberId, MIN(YEAR(DispenseDate)) AS DispenseYear, 
                        sum( case when NdcDescription like '", meds, "%' then 1 else 0 end) as n_", groups[i],",")
  
  otherMeds <- medInputList[ ! medInputList$Group %in% groups[i], ]
  otherGroups <- unique(otherMeds$Group)
  for( j in 1:length(otherGroups)){
    print(j)
    otherMedsList <- paste( tolower(otherMeds[ otherMeds$Group == otherGroups[j], "medicationName"]), collapse="%' OR LOWER(NdcDescription)  like '%")
    if( j == 1){
      queryContinue <- paste0(queryStart, "sum( case when NdcDescription like '", otherMedsList, "%' then 1 else 0 end) as n_", otherGroups[j],",")
    }else if(j < length(otherGroups)){
      queryContinue <- paste0(queryContinue, "sum( case when NdcDescription like '", otherMedsList, "%' then 1 else 0 end) as n_", otherGroups[j],",")
    }else{
      queryEnds <- paste0(queryContinue, "sum( case when NdcDescription like '", otherMedsList, "%' then 1 else 0 end) as n_", otherGroups[j], 
                          " INTO ", groups[i], "_only from PharmacySubsetTest2014
                          GROUP BY MemberId having 
                          sum( case when NdcDescription like '", meds, "%' then 1 else 0 end) > 0 and ")
      
    }
  }
  for( w in 1:length(otherGroups)){
    print(w)
    otherMedsList <- paste( tolower(otherMeds[ otherMeds$Group == otherGroups[w], "medicationName"]), collapse="%' OR LOWER(NdcDescription)  like '%")
    if( w == 1){
      finalQuery <- paste0(queryEnds, "sum( case when NdcDescription like '", otherMedsList, "%' then 1 else 0 end) = 0 and ")
    }else if(w < length(otherGroups)){
      finalQuery <- paste0(finalQuery, "sum( case when NdcDescription like '", otherMedsList, "%' then 1 else 0 end) = 0 and ")
    }else{
      finalQuery <- paste0(finalQuery, "sum( case when NdcDescription like '", otherMedsList, "%' then 1 else 0 end) = 0")
    }
  }
  
  dbSendUpdate( cn, paste0("DROP TABLE IF EXISTS ", groups[i], "_only"))
  dbSendUpdate( cn, finalQuery)
  
}

#### Create the ACTMap3 and ACTlevel3_CM_3Times tables

In [ ]:
dbSendUpdate( cn, "drop table if exists ACTMap3")

dbSendUpdate( cn, "SELECT ASD.MemberId, F.DateServiceStarted, ACT.Level3 
              INTO ACTMap3 
              FROM 
              ASDMembers ASD 
              INNER JOIN FactIcd F ON 
              F.MemberId = ASD.MemberId 
              INNER JOIN ACT_codes ACT ON 
              ACT.IcdCode = F.Icd 
                        WHERE YEAR(F.DateServiceStarted) >= 2012 
                        GROUP BY ASD.MemberId, DateServiceStarted, Level3")
 
#only include individuals with comorbidities diagnosed >= 3 times 
dbSendUpdate( cn, "drop table if exists ACTLevel3_CM_3Times")

dbSendUpdate( cn, "SELECT MemberId, Level3, COUNT(*) as Level3_counts  
INTO ACTLevel3_CM_3Times 
FROM ACTMap3 
GROUP BY MemberID, Level3 
HAVING COUNT(Level3) >= 3")

## Subset patients taking only one drug
Subset those that are only taking one of the drugs and extract the total number of patients in each group, that will be used later to estimate the prevalence of each comorbidity.



In [ ]:
for( i in 1:length(groups)){
  print( groups[i] )
  
  # individuals in cohort taking ONLY one drug
  print( dbGetQuery( cn, paste0( "SELECT COUNT(DISTINCT MemberId) FROM ", groups[i],"_only")))
  print( "####")
  
  #extract all the comorbidities from those patients
  allComorb <- dbGetQuery( cn, paste0("SELECT A.MemberId, A.Level3, YEAR(A.minDate) as minDatee FROM ACTLevel3_CM_3Times A
                                        INNER JOIN ", 
                                        groups[i],"_only G ON 
                                        A.MemberId = G.MemberId"))
                                        
  #extract all comorbidities before the first time drug was prescribed                                     
  allComorbBefore <- dbGetQuery( cn, paste0("SELECT A.MemberId, A.Level3, YEAR(A.minDate) as minDatee FROM ACTLevel3_CM_3Times A
                                        INNER JOIN ", 
                                        groups[i],"_only G ON 
                                        A.MemberId = G.MemberId 
                                        WHERE YEAR(A.minDate) < G.DispenseYear"))
                                        
  #estimate the prevalence of each comorbidity ans save it into a table
  prevalenceAll <- as.data.frame( table( allComorb$Level3))
  colnames(prevalenceAll) <- c("Level3","Level3_prevalence")
  dbSendUpdate( cn, paste0( "DROP TABLE IF EXISTS ", groups[i],"_CMs_all"))
  dbWriteTable( cn, paste0(groups[i],"_CMs_all"), prevalenceAll, row.names = FALSE)

  #estimate the prevalence of each comorbidity ans save it into a table
  prevalenceBefore <- as.data.frame( table( allComorbBefore$Level3))
  colnames(prevalenceBefore) <- c("Level3","Level3_prevalence")
  dbSendUpdate( cn, paste0( "DROP TABLE IF EXISTS ", groups[i],"_CMs_before"))
  dbWriteTable( cn, paste0(groups[i],"_CMs_before"), prevalenceBefore, row.names = FALSE)
    
}

##  Heatmap representation: all comorbidities

In [ ]:
inputData <- as.data.frame( matrix(ncol=3, nrow=length(groups)))
colnames(inputData) <- c("Drug", "n", "tableName")


for( i in 1:length(groups)){
  inputData$Drug[i] <- groups[i]
  inputData$n[i] <- dbGetQuery( cn, paste0( "SELECT COUNT(DISTINCT MemberId) FROM ", groups[i],"_only"))
  inputData$tableName[i] <- paste0( groups[i],"_CMs_all")
}

for( i in 1:nrow( inputData)){
  queryCounts <- paste0( "SELECT * FROM ", inputData$tableName[i], 
                         " ORDER BY Level3_prevalence DESC")
  print( i )
  if( i == 1){
    outputAll <- dbGetQuery( cn, queryCounts )
    outputAll$drug <- inputData$Drug[i]
    outputAll$totalPatients <- inputData$n[i]
  }else{
    intermediateOutputAll <- dbGetQuery( cn, queryCounts )
    intermediateOutputAll$drug <- inputData$Drug[i]
    intermediateOutputAll$totalPatients <- inputData$n[i]
    outputAll <- rbind( outputAll, intermediateOutputAll )
  }
}

##  Heatmap representation: all comorbidities before first time drug was prescribed 

In [ ]:
inputData <- as.data.frame( matrix(ncol=3, nrow=length(groups)))
colnames(inputData) <- c("Drug", "n", "tableName")


for( i in 1:length(groups)){
  inputData$Drug[i] <- groups[i]
  inputData$n[i] <- dbGetQuery( cn, paste0( "SELECT COUNT(DISTINCT MemberId) FROM ", groups[i],"_only"))
  inputData$tableName[i] <- paste0( groups[i],"_CMs_before")
}

for( i in 1:nrow( inputData)){
  queryCounts <- paste0( "SELECT * FROM ", inputData$tableName[i], 
                         " ORDER BY Level3_prevalence DESC")
  print( i )
  if( i == 1){
    outputBefore <- dbGetQuery( cn, queryCounts )
    outputBefore$drug <- inputData$Drug[i]
    outputBefore$totalPatients <- inputData$n[i]
  }else{
    intermediateOutputBefore <- dbGetQuery( cn, queryCounts )
    intermediateOutputBefore$drug <- inputData$Drug[i]
    intermediateOutputBefore$totalPatients <- inputData$n[i]
    outputBefore <- rbind( outputBefore, intermediateOutputBefore )
  }
}

### Percentage of patients with each comorbidity
We estimate the percentage of patients with each comorbidities and we do a first subset selecting only those comorbidities that are in at least 10% of the patients. We mapp to the ACT levels, to later aggregate by a higher category if need it, and we remove some comorbidities that are not considered as clinically relevant for this study. 

In [ ]:
output <- outputAll
#output <- outputBefore

output$totalPatients <- as.numeric( output$totalPatients )
output$percentage <- round( 100*(output$Level3_prevalence / output$totalPatients), 3)

#select only those comorbidities in at least 1% of the patients
outputSubset <- output[ output$percentage >= 1, ]

#map to act
actMapping <- dbGetQuery( cn, "SELECT Level1, Level3 FROM ACT_ICD10_ICD9_3")
actMapping <- actMapping[!duplicated( actMapping), ]
actMapping <- actMapping[!duplicated( actMapping$Level3 ), ]

#mapped the level3 to level1
outputMapped <- merge( outputSubset, actMapping)

#exclude the comorbidities that are not clinically relevant
excludedGroups <- c('Autistic disorder',
                      'Encounter for newborn, infant and child health examinations',
                      'motorized bicycle',
                      'Other unknown and unspecified cause of morbidity or mortality',
                      'Need for prophylactic vaccination and inoculation, Influenza',
                      'Bus occupant injured in transport accident (v70-v79)',
                      'Encounter for other specified aftercare',
                      'Other long term (current) drug therapy',
                      'Body mass index (bmi) pediatric',
                      'Pharyngitis (acute) nos',
                      'Acute upper respiratory infection, unspecified',
                      'Acne vulgaris',
                      'Hyperlipidemia, unspecified',
                      'Encounter for adult periodic examination (annual) (physical) and any associated laboratory and radiologic examinations'
                      )

outputMapped <- outputMapped[! outputMapped$Level3 %in%  excludedGroups, ]
save(outputMapped, file = "outputMapped.RData")

### Plot the heatmap

In [ ]:
 #plot the heatmap
  htmpComorbBefore <- ggplot(outputMapped, aes(drug, Level3, fill= percentage)) +
    geom_tile()+
    ggplot2::theme_bw() +
    ggplot2::theme(axis.text.x = ggplot2::element_text(angle = 45, hjust = 1),
                   panel.grid = element_blank(),
                   text = ggplot2::element_text(size = 6),
                   axis.title = ggplot2::element_text(size = 6))
  save(htmpComorbBefore, file = "./htmpComorbBefore.RData")
  ### plot in R studio (out from o2)
  load( "./outputMapped.RData")

  #remove Autistic disorder (sanity check, all patients should have ASD)
  toplot <- outputMapped[! outputMapped$Level3 %in% c("Autistic disorder", "Asperger's syndrome"), ]
  #select 10% to filter
  drugs <- unique(toplot$drug)
  for( i in 1:length(drugs)){
    selection <- toplot[ toplot$drug == drugs[i] &
                           toplot$percentage > 10, ]
    if(i == 1){
      phenoList <- selection$Level3
    }else{
      subSet <- selection$Level3
      phenoList <- unique( c( phenoList, subSet))
    }
  }

  toplot <- toplot[ toplot$Level3 %in% phenoList, ]
  toplot <- toplot[ order(toplot$Level1), ]

  # sort/display the groups based on groups
  groups <- as.character( unique( medInputList$Group))
  toplot$drug <- factor(toplot$drug, levels=groups)
  # they should be same!
  stopifnot(sort(unique(outputMapped$drug))==sort(groups))
  
  # unify the names of the comorbidities removing the ICD original code 
  toplot$Level3[ toplot$Level3 == "(K59.0) Constipation"] <- "Constipation"
  toplot$Level3[ toplot$Level3 == "(G47.0) Insomnia"] <- "Insomnia"
  
  # cut the name of the comorbidity to the first 60 characters
  toplot$Level3 <- stringr::str_wrap(toplot$Level3, 60)
  
  # sort/display the comorbidities based on Level 1 ACT ontology 
  act_level <- as.character( unique( toplot$Level3))
  toplot$Level3 <- factor(toplot$Level3, levels=act_level)

  #create the heatmap
  htmpOutput<- ggplot(toplot, aes(drug, Level3, fill= percentage)) + # 60
    geom_tile()+
    scale_fill_gradient(low="white", high="blue") +
    #scale_fill_distiller(palette = "YlOrRd")+
    #scale_fill_continuous(low="#F7FBFF", high="#2171B5", name="Events")+
    ggplot2::theme_bw() +
    ggplot2::theme(axis.text.x = ggplot2::element_text(angle = 45, hjust = 1,face="bold"),
                   panel.grid = element_blank(),
                   axis.text.y = ggplot2::element_text(size=rel(0.9)),
                   axis.title = ggplot2:: element_text(size=rel(1.05)))+
    labs(title = NULL, x = "", y =  "",fill="Percentage")

  htmpOutput

  write.table( toplot, file="heatmap_values.txt", 
             col.names = TRUE, row.names = FALSE, quote = FALSE, sep = "\t")

  ggsave(filename="htmpOutput_L3.png", plot=htmpOutput, device="png",
         height=11, width=14, units="in", dpi=500)

 htmpOutput2 <- htmpOutput +  scale_y_discrete(position = "right") + 
   facet_grid( vars( stringr::str_wrap(Level1, 40)), scales = "free", space = "free",switch = "y") +
   theme(strip.text.y.left = element_text(angle = 0,size=rel(1.0))) 

  htmpOutput2
  ggsave(filename="htmpOutputL3_L1.png", plot=htmpOutput2, device="png",
         height=12.5, width=15, units="in", dpi=500)
